#### 前言
<p> 大数据时代下，互联网与各行各业的距离逐渐拉近，物联网化也逐渐成为实体行业新的趋势，数据分析这一职位也受到了越来越多的实体行业的关注，本文爬取猎聘网站上上海的汽车行业数据分析相关职位的招聘信息，并对爬取的数据进行了简单的数据分析。

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [ ]:
#登陆猎聘网站，找到职位搜索
#地点选择上海
#行业选择汽车行业
#搜索关键字：数据分析
#得到以下url
url = "https://www.liepin.com/zhaopin/?pubTime=&ckid=111f610f1f2be392&fromSearchBtn=2&compkind=&isAnalysis=&init=-1&searchType=1&dqs=020&industryType=industry_06&jobKind=&sortFlag=15&degradeFlag=0&industries=350&salary=&compscale=&clean_condition=&key=%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90&headckid=111f610f1f2be392&d_pageSize=40&siTag=ZFDYQyfloRvvhTxLnVV_Qg%7EvyYl3W7OHISXWcwZHBOQVA&d_headId=f85576d781505ba50bfad203dc61e60b&d_ckId=f85576d781505ba50bfad203dc61e60b&d_sfrom=search_prime&d_curPage=1&curPage={}"

In [ ]:
#爬取关键信息的函数
def get_recruit_details(recruit):
    """
    爬取单个招聘信息的函数
    爬取的信息如下
    公司名称、所在区与、职位名称、发布日期、工资待遇、聘用资格、公司福利（标签）
    """
    result = {}
    res = requests.get(recruit)
    soup = BeautifulSoup(res.text, "html.parser")
    
    result["company"] = soup.select(".title-info h3")[0].text.strip() #公司名称
    result["zone"] = soup.select(".basic-infor span")[0].text.strip() #公司所在区
    result["position"] = soup.select(".title-info h1")[0].text.strip() #职位名称
    result["release_date"] = soup.select(".basic-infor time")[0]["title"] #发布日期
    result["salary"] = soup.select(".job-title-left p")[0].text.strip() #工资待遇
   
    #聘用资格 由于部分网页使用class名称不一样加上本人水平有限，故写了两个for循环来爬取该信息
    result["qualifications"] = []                               
    for qualification in soup.select(".resume.clearfix span"):
            result["qualifications"].append(qualification.text)
    
    for qualification in soup.select(".job-qualifications span"):
            result["qualifications"].append(qualification.text)

    #公司福利（标签）    
    result["labels"] = []
    for label in soup.select(".comp-tag-box span"):
        result["labels"].append(label.text)
    
    result["context"] = soup.select(".content.content-word")[0].text.strip()
    
    return result

In [ ]:
#爬取一个网页上所有招聘信息函数
def get_one_page_info(p_url):
    """
    爬取一个网页上的所有招聘信息
    """
    p_res = requests.get(p_url)
    p_soup = BeautifulSoup(p_res.text, "html.parser")
    
    #页面上每一个招聘信息的url有区别，故提取出完整的url放置在nets列表里
    nets = []
    for i in p_soup.select(".job-info"):
        head = "https://www.liepin.com" 
        body = i.a["href"]+"?"+i.a["data-promid"]
        if head not in body:
            body = head + body
        nets.append(body)  
        
    #由于部分招聘页面所用的标签名称，格式有区别，不能用之前编写好的函数爬取（主要原因是本人能力有限TOT），故放弃该网址的爬取，将其url打印出来单独查看
    res = []
    for net in nets:
        try:
            res.append(get_recruit_details(net))
        except IndexError:
            print("TOT")
            print(net)
    
    return res

In [ ]:
#一共有30页，我们将所有信息爬取下来放进total里，并打印出不能爬取的页面
total = [] 
for page in range(30):
    one_page = get_one_page_info(url.format(page))
    total.extend(one_page)
    print("page", page+1, "is done")

In [ ]:
#存储数据
df = pd.DataFrame(total)
today = datetime.datetime.strftime(datetime.date.today(), "%Y%m%d")
df.to_excel(today+"liepin.xlsx")